In [1]:
import csv

def get_borehole_data():
    with open('Boreholes_Boyabu.csv','r') as csvfile:
        reader = csv.reader(csvfile, delimiter = ',',quotechar = '"')
        for row in reader:
            yield row
            
borehole_data = get_borehole_data()

columns = next(borehole_data)
water_sources = list(borehole_data)

borehole_coordinates = [item[8:10][::-1] for item in water_sources]
borehole_coordinates = [(float(i[0]),float(i[1])) for i in borehole_coordinates]

In [2]:
from util.distances import distance_filter
from json import loads, dumps

m500 = 0.310686
m200 = 0.124274

# calculate number of boreholes serving each tent

with open('LAT3D73400_LON18D76178_RANGE25D000.json','r') as file:
    
    geojson = loads(file.read())
    
    for i in range(len(geojson['features'])):
        feature = geojson['features'][i]
        within_emergency_range = 0
        within_post_emergency_range = 0
        for borehole_coordinate in borehole_coordinates:
            if distance_filter(borehole_coordinate,feature['geometry']['coordinates'],m500):
                within_emergency_range += 1
            if distance_filter(borehole_coordinate,feature['geometry']['coordinates'],m200):
                within_post_emergency_range += 1
        
        geojson['features'][i]['500m range'] = within_emergency_range
        geojson['features'][i]['200m range'] = within_post_emergency_range
        
        
with open('LAT3D73400_LON18D76178_RANGE25D000_weighted.json','w') as file:
    file.write(dumps(geojson))

    
# write separate geojsons 
with open('LAT3D73400_LON18D76178_RANGE25D000_weighted.json','r') as file:
    geojson = loads(file.read())
    features = geojson['features']
    range500 = []
    range200 = []
    range000 = []
    for feature in features:
        if feature['500m range'] != 0:
            range500 += [feature]
        if feature['200m range'] != 0:
            range200 += [feature]
        if feature['500m range'] == 0:
            range000 += [feature]
    for features,fname in zip([range500,range200,range000], ['LAT3D73400_LON18D76178_RANGE25D000_%i.json' % i for i in (500,200,0)]):
        with open(fname,'w') as file_:
            geojson['features'] = features
            file_.write(dumps(geojson))

In [3]:
# calculate number of tents served by each borehole
import csv

borehole_data = list(get_borehole_data())
borehole_data[0] = borehole_data[0] + ['Tents served 500m','Tents served 200m']

m500 = 0.310686
m200 = 0.124274

with open('LAT3D73400_LON18D76178_RANGE25D000_weighted.json','r') as file:
    geojson = loads(file.read())

with open('Boreholes_Boyabu_weighted.csv','w') as csvfile:
    
    csvwriter = csv.writer(csvfile, delimiter=' ',quotechar='"')
    csvwriter.writerow(borehole_data[0])
    
    for i in range(1,len(borehole_data[1:])):
        borehole = borehole_data[i]
        borehole_coordinate = borehole[8:10][::-1]
        borehole_coordinate = (float(borehole_coordinate[0]),float(borehole_coordinate[1]))

        tents_within_500 = 0
        tents_within_200 = 0

        for feature in geojson['features']:
            if distance_filter(borehole_coordinate,feature['geometry']['coordinates'],m500):
                tents_within_500 += 1/feature['500m range']
            if distance_filter(borehole_coordinate,feature['geometry']['coordinates'],m200):
                tents_within_200 += 1/feature['200m range']

        borehole_data[i] += [tents_within_500,tents_within_200]
        csvwriter.writerow(borehole_data[i])